# Creating a simple chatbot using the ChatGPT model
This sample notebook demonstrates  creation of a simple chatbot that uses the ChatGPT model using the Chat Completion API.  The API takes in an array of messages that have the following parts:
- role (system - sets the overall behavior for the assistant , user - provides specific instructions for the assistant, assistant - stores previous responses and can be used for fine tuning to achieve desired behavior)
- content

For this lab the system message will be a simple description, and the messages sent to the api will be a collection that inclues all of the previous messages and responses.  In addition this example will track the number of tokens in the message to ensure it does not exceed the maximum.


In [ ]:
# import os module & the OpenAI Python library for calling the OpenAI API
# please make sure you have installed required libraries via pip install -r requirements.txt
import os
import openai
from dotenv import load_dotenv

load_dotenv()



In [ ]:
# Load config values

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = os.getenv("OPENAI_API_BASE")
    
# This is set to `azure`
openai.api_type = "azure"

# The API key for your Azure OpenAI resource.
openai.api_key = os.getenv("OPENAI_API_KEY")



# Setting up the deployment name
chatgpt_model_name = os.getenv("CHATGPT_MODEL")

# Currently Chat Completions API have the following versions available: 2023-03-15-preview
openai.api_version = os.getenv("OPENAI_API_VERSION")


## 1.0 Define Request to OpenAI API
For these labs we are using the Azure OpenAI Chat Completion API, this is different than the Completions API.  Information about its usage can be found here:
(https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference#chat-completions)
The chat completions API takes a collection of messages to provide history and context of chat to the model.  This function takes a collection of messages which are objects that define a role and content, if the role is the user role the message also contains name as part of the object.

We also create a function that calculates the number of tokens received from the message collection so that we can keep track of message size and not exceed threshhold.


In [ ]:
import tiktoken

# Creating base message 
base_system_message = "You are a helpful assistant with a cheerful personality"

system_message = f"{base_system_message.strip()}"
print(system_message)

# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(messages, model_name, max_response_tokens=500):
    response = openai.ChatCompletion.create(
        engine=chatgpt_model_name,
        messages=messages,
        temperature=0.5,
        max_tokens=max_response_tokens,
        top_p=0.9,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response['choices'][0]['message']['content']

# Defining a function that counts the number of tokens
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

## 2.0 The conversation

In [ ]:
# This is the first user message that will be sent to the model. Feel free to update this.
max_response_tokens = 500
token_limit=2048
name = input('Enter Your Name: ')
print ('Welcome to Chatbot Service! Let me know how can I help you')
conversation=[
    {"role": "system", "content": system_message},
]
while True:
    request = input(name+':')
    conversation.append({"role": "user", "name":"example_user", "content":request})
    conv_history_token_count = num_tokens_from_messages(conversation)
    #reduce history to fall below limits
    while (conv_history_token_count+max_response_tokens >= token_limit):
        del conversation[1] 
        conv_history_token_count = num_tokens_from_messages(conversation)
    if request=="Bye" or request=='bye':
        print('Bot: Bye')
        break
    else:
        response = send_message(conversation, chatgpt_model_name, max_response_tokens)
        conversation.append({"role": "assistant", "content":response})
        print('sent token count: ' + str(conv_history_token_count))
        print('Bot: ', response)